In [65]:
#importing pandas and numpy library for reading files and data manupulation
# import re i.r regular expression

import pandas as pd
import numpy as np
import re 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # access convergence dint work
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import accuracy_score




In [66]:
#Read the train and test files

train_data = pd.read_csv('C:/Users/punit/Binary Prediction of Smoker Status using Bio-Signals/train.csv')
test_data = pd.read_csv('C:/Users/punit/Binary Prediction of Smoker Status using Bio-Signals/test.csv')

In [67]:
# rename the columns

train_data = train_data.rename(
                        
            columns = { 
                       old: re.sub(r'\s|\)|\(','_', old).lower()
                       for old in train_data.columns
                      }
)

# rename the columns

test_data = test_data.rename(
                        
            columns = { 
                       old: re.sub(r'\s|\)|\(','_', old).lower()
                       for old in test_data.columns
                      }
)

In [68]:
#drop columns not needed to build the model.
# in this case dropping id

x_train = train_data.drop('id', axis = 1)
x_test = test_data.drop('id', axis = 1) 


#drop target column from train data and assign target variable to val set

x_train = x_train.drop('smoking', axis = 1)
y_train = train_data['smoking']




In [69]:
# split the data

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,train_size = 0.8, test_size = 0.2 , random_state = 0 )


In [119]:

# Create and train the XGBoost model
model = xgb.XGBClassifier(n_estimators= 1000, learning_rate=0.08, max_depth = 5, gamma = 0,
                          reg_lambda = 5, subsample = 1, colsample_bytree = 1, min_child_weight = 1,
                          objective = 'binary:logistic', nthread = 4)
model.fit(x_train, y_train)

# Make predictions on training data
y_pred = model.predict(x_val)

# Calculate the AUC score for train data
train_auc = roc_auc_score(y_val, y_pred)

print("AUC score for train data is:", train_auc)


AUC score for train data is: 0.7908863376884036


In [121]:
# Make predictions on test data
y_pred_test = model.predict(x_test)

# Create a pandas DataFrame of the test data and predictions
test_df = pd.DataFrame({'id': x_test.index, 'smoking': y_pred_test})

# Save the DataFrame to a CSV file
test_df.to_csv('submission.csv', index=False)
